# 使用PaddleClas进行AI艺术鉴赏预测
1. 下载好数据集--可以再下载PaddleClas进行相关训练 -- 数据集名称：AI艺术鉴赏挑战赛 - 看画猜作者  （已经传到AIStudio上）
2. 修改学习率调整机制、网络模型等进行再训练,以求最高的精度

In [1]:
import numpy as np           
import pandas as pds         
from matplotlib import pyplot as plt    
import paddle.fluid as fluid
import paddle.fluid.dygraph.nn as nn
from paddle.fluid import dygraph
from paddle.fluid.dygraph import Layer
from paddle.fluid.dygraph.nn import Linear, Conv2D, Pool2D, BatchNorm, LayerNorm, Dropout
from paddle.fluid.layers import resize_nearest, flatten              # 向上采样
from PIL import Image
from math import ceil            # ceil 是一个向上取整

import os
import zipfile as zipf
import random      

random.seed(10)      # 随机数种子
np.random.seed(10)   # 随机数种子

# 一、查看数据集长度和分类情况

In [2]:
# 加载指定的csv训练集文件--得到训练标签以及数据集长度信息
def csv_load(csv_path = 'work/datasets/train.csv'):
    
    csv_data = pds.read_csv(csv_path)    # pandas读取csv文件
    csv_data = pds.DataFrame(csv_data)   # 转换为DataFrame格式
    csv_data = np.array(csv_data)        # 将pandas.DataFrame转化未numpy数据格式
    
    # 数据集由图片文件id+对应的label组成
    image_ids, lable_ids = csv_data[:,0], csv_data[:,1]    # 将数据划分--narray数据
    length_img = len(image_ids)                            # 图片数量--训练集长度
    range_label = [lable_ids.min(), lable_ids.max()]       # 标签范围

    print("The datasets of length is {0}.".format(length_img))   # 打印信息
    print("The labels of range is {0}.".format(range_label))

    return image_ids, lable_ids

# 执行数据信息获取
image_id, label_id = csv_load()  # 49个类别

The datasets of length is 7227.
The labels of range is [0, 48].


# 二、划分数据 -- 由于只有一个训练集，需要人为手动划分数据集--提供eval

In [3]:
def data_divide(image_id=[], label_id=[],train_percen=0.7, eval_percen=0.2, test_percen=0.1):
    '''
        默认以为：0.7， 0.2， 0.1划分数据集
    '''
    rate = train_percen + eval_percen + test_percen  # 总占比
    assert rate <= 1.0 and rate >=rate, \
            "--train_percen({0:.2f}) + eval_percen({1:.2f}) + test_percen({2:.2f}) = {3:.2f}, however it is not 1!".format(train_percen, eval_percen, test_percen, train_percen + eval_percen + test_percen)

    lens = len(image_id)  # 数据集长度

    # 各部分索引范围
    indexs_train_start = 0
    indexs_train_end = int(lens * train_percen)
    indexs_eval_start = indexs_train_end
    indexs_eval_end = int(lens * (train_percen+eval_percen))
    indexs_test_start = indexs_eval_end
    indexs_test_end = lens
    
    # 不需要打乱
    shuffle_indexs = [i for i in range(lens)]   # 设置数据集index
    # np.random.shuffle(shuffle_indexs)          # 打乱index--然后按index取数据即可
    
    train_data = (image_id[shuffle_indexs[indexs_train_start:indexs_train_end]], \
                    label_id[shuffle_indexs[indexs_train_start:indexs_train_end]])

    eval_data = (image_id[shuffle_indexs[indexs_eval_start:indexs_eval_end]], \
                    label_id[shuffle_indexs[indexs_eval_start:indexs_eval_end]])
    test_data = (image_id[shuffle_indexs[indexs_test_start:indexs_test_end]], \
                    label_id[shuffle_indexs[indexs_test_start:indexs_test_end]])
    
    return train_data, eval_data, test_data


In [4]:
# 执行数据集划分
train_datas, eval_datas, test_datas = data_divide(image_id=image_id, label_id=label_id, train_percen=0.8, eval_percen=0.2, test_percen=0.0)
train_img_id , train_lab_id = train_datas
eval_img_id , eval_lab_id = eval_datas
test_img_id , test_lab_id = test_datas  # 无test数据
print(train_img_id[0:10])
print(train_lab_id[0:10])

[0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 3 2 4 5 6 7]


# 三、产生paddle_clas需要的文本索引

1. train.txt -- 包含划分好的数据集id，以及分类序号
2. eval.txtt -- 包含划分好的数据集id，以及分类序号

In [5]:
# 给获得的数据进行拼接--image 《==》 label
train_data = np.concatenate((np.array(train_img_id).reshape((-1, 1)) , np.array(train_lab_id).reshape((-1, 1))) , axis=-1).astype('str')
eval_data = np.concatenate((np.array(eval_img_id).reshape((-1, 1)) , np.array(eval_lab_id).reshape((-1, 1))) , axis=-1).astype('str')

# 给训练数据序号添加图片类型后缀
for i in range(len(train_data)):
    train_data[i, 0] = str(train_data[i, 0] + r'.jpg')
# 给eval保存
for i in range(len(eval_data)):
    eval_data[i, 0] = str(eval_data[i, 0] + r'.jpg')

# 将训练数据文本保存
train_data = pds.DataFrame(train_data)
train_data.to_csv('work/datasets/train_clas.txt', sep=' ', header=0, index=0)    # 保存paddle-cls需要的数据索引文本
# 将eval数据文本保存
eval_data = pds.DataFrame(eval_data)
eval_data.to_csv('work/datasets/eval_clas.txt', sep=' ', header=0, index=0)    # 保存paddle-cls需要的数据索引文本

# 四、开始使用PaddleClas进行训练


In [1]:
# 跳转工作路径
%cd /home/aistudio/PaddleClas/

/home/aistudio/PaddleClas


In [9]:
# 下载需要的依赖项 -- 第一次配置环境需要下载
# !pip install --upgrade pip
# !pip install --upgrade -r requirements.txt  # 能使用就不需要再操作了

%env PYTHONPATH=.:$PYTHONPATH  # 配置python解释器路径
%env CUDA_VISIBLE_DEVICES=0    # 配置CUDA驱动号

/home/aistudio/PaddleClas
env: PYTHONPATH=.:$PYTHONPATH  # 配置python解释器路径
env: CUDA_VISIBLE_DEVICES=0    # 配置CUDA驱动号


In [4]:
!export CUDA_VISIBLE_DEVICES=0       #设置GPU

In [ ]:
# 下载预训练模型 -- 根据需要进行修改
# !python tools/download.py -a ResNet50_ACNet_deploy -p ./pretrained -d True
# !python tools/download.py -a Res2Net50_26w_4s -p ./pretrained -d True
# !python tools/download.py -a ResNeXt152_vd_32x4d -p ./pretrained -d True 
# !python tools/download.py -a EfficientNetB4	 -p ./pretrained -d True 
# !python tools/download.py -a EfficientNetB3	 -p ./pretrained -d True 
# !python tools/download.py -a ResNet50_ACNet_deploy -p ./pretrained/ -d True
!python tools/download.py -a ResNeXt101_32x8d_wsl -p ./pretrained/ -d True

In [ ]:
# 注意查看yml模型文件中的训练集路径以及验证集路径--及时修改成自己的数据集路径
#EfficientB4
!python -m paddle.distributed.launch \
    tools/train.py \
        -c ./configs/EfficientNet/EfficientNetB4.yaml \
        # -o pretrained_model=./output/EfficientNetB4/best_model/ppcls    #基于已训练的部分继续训练

In [ ]:
#ResNeXt101_32x8d_wsl
!python -m paddle.distributed.launch tools/train.py -c ./configs/ResNeXt101_32x8d_wsl.yaml\
-o pretrained_model=./output/ResNeXt101_32x8d_wsl/best_model/ppcls    #基于已训练的部分继续训练

In [ ]:
#对ResNeXt101_32x8d_wsl更换策略继续训练
!python -m paddle.distributed.launch tools/train.py -c ./configs/ResNeXt101_32x8d_wsl-2.yaml -o pretrained_model=./output/ResNeXt101_32x8d_wsl/best_model/ppcls    #基于已训练的部分继续训练

In [ ]:
#EfficientNetB3
!python -m paddle.distributed.launch tools/train.py -c ./configs/EfficientNet/EfficientNetB3.yaml -o pretrained_model=./output/EfficientNetB3/best_model/ppcls    #基于已训练的部分继续训练

-----------  Configuration Arguments -----------
cluster_node_ips: 127.0.0.1
log_dir: None
log_level: 20
node_ip: 127.0.0.1
print_config: True
selected_gpus: None
started_port: None
training_script: tools/train.py
training_script_args: ['-c', './configs/EfficientNet/EfficientNetB3.yaml', '-o', 'pretrained_model=./output/EfficientNetB3/best_model/ppcls']
use_paddlecloud: False
------------------------------------------------
INFO 2021-11-19 16:24:33,938 launch.py:210] get cluster from args:job_server:None pods:['rank:0 id:None addr:127.0.0.1 port:None visible_gpu:[] trainers:["gpu:[\'0\'] endpoint:127.0.0.1:52857 rank:0"]'] job_stage_flag:None hdfs:None
INFO 2021-11-19 16:24:33,940 utils.py:367] start trainer proc:['/opt/conda/envs/python35-paddle120-env/bin/python', '-u', 'tools/train.py', '-c', './configs/EfficientNet/EfficientNetB3.yaml', '-o', 'pretrained_model=./output/EfficientNetB3/best_model/ppcls'] env:{'FLAGS_selected_gpus': '0', 'PADDLE_TRAINER_ID': '0', 'PADDLE_CURRENT_ENDPO

In [3]:
# 验证数据
#1.EfficientNetB4
# pretrained_model -- 训练好的模型（非导出的预测模型）
# !python -m paddle.distributed.launch \
#     tools/eval.py \
#     -c ./configs/eval.yaml \
#     -o ARCHITECTURE.name="EfficientNetB4" \
#     -o pretrained_model=./output/EfficientNetB4/best_model/ppcls
#2.ResNeXt101_32x8d_wsl
!python -m paddle.distributed.launch \
    tools/eval.py \
    -c ./configs/eval-EfficientNetB3.yaml \
    -o ARCHITECTURE.name="EfficientNetB3" \
    -o pretrained_model=./output/EfficientNetB3/best_model/ppcls

-----------  Configuration Arguments -----------
cluster_node_ips: 127.0.0.1
log_dir: None
log_level: 20
node_ip: 127.0.0.1
print_config: True
selected_gpus: None
started_port: None
training_script: tools/eval.py
training_script_args: ['-c', './configs/eval-EfficientNetB3.yaml', '-o', 'ARCHITECTURE.name=EfficientNetB3', '-o', 'pretrained_model=./output/EfficientNetB3/best_model/ppcls']
use_paddlecloud: False
------------------------------------------------
INFO 2021-11-19 10:18:02,579 launch.py:210] get cluster from args:job_server:None pods:['rank:0 id:None addr:127.0.0.1 port:None visible_gpu:[] trainers:["gpu:[\'0\'] endpoint:127.0.0.1:44837 rank:0"]'] job_stage_flag:None hdfs:None
INFO 2021-11-19 10:18:02,580 utils.py:367] start trainer proc:['/opt/conda/envs/python35-paddle120-env/bin/python', '-u', 'tools/eval.py', '-c', './configs/eval-EfficientNetB3.yaml', '-o', 'ARCHITECTURE.name=EfficientNetB3', '-o', 'pretrained_model=./output/EfficientNetB3/best_model/ppcls'] env:{'FLAGS_se

# 五、训练结束后，对模型文件的处理

1. 首先在保存文件前，将best模型取出，再放回 -- 然后执行模型的保存
2. 将保存的模型，从默认路径移动到指定目录下 -- 方便使用和保存

In [15]:
# # 1.0将训练好的best模型移出 -- 删除原来的文件夹 -- 只需修改相应的模型路径即可
# # /home/aistudio/PaddleClas/output/ResNeXt152_vd_32x4d/best_model  中的 ResNeXt152_vd_32x4d 换成自己的模型文件夹即可
# !mv /home/aistudio/PaddleClas/output/ResNeXt101_32x8d_wsl/best_model /home/aistudio/PaddleClas/output/best_model
# !mv /home/aistudio/PaddleClas/output/ResNeXt101_32x8d_wsl
# !mkdir /home/aistudio/PaddleClas/output/ResNeXt101_32x8d_wsl

# # 1.1将训练好的best模型放回 -- 只需修改相应的模型路径即可
# !mv /home/aistudio/PaddleClas/output/best_model /home/aistudio/PaddleClas/output/ResNeXt101_32x8d_wsl/best_model

mv: 在'/home/aistudio/PaddleClas/output/ResNeXt101_32x8d_wsl' 后缺少了要操作的目标文件
Try 'mv --help' for more information.
mkdir: 无法创建目录"/home/aistudio/PaddleClas/output/ResNeXt101_32x8d_wsl": 文件已存在


In [16]:
# 模型保存 -- PaddleClas主目录里
# ResNeXt101_32x8d_wsl -- 改成自己的模型文件夹
!python tools/export_model.py \
    --model=ResNeXt101_32x8d_wsl \
    --pretrained_model=./output/ResNeXt101_32x8d_wsl/best_model/ppcls \
    --output_path=./models/ResNeXt101_32x8d_wsl/

In [11]:
# # 2.0创建模型保存文件夹
# !mkdir /home/aistudio/PaddleClas/models/ResNeXt101_32x8d_wsl
# # 2.1模型文件转移 -- /home/aistudio/PaddleClas/model -- 为本次训练目标的不同预测模型的保存点
# !mv /home/aistudio/PaddleClas/model /home/aistudio/PaddleClas/models/ResNeXt101_32x8d_wsl/model
# !mv /home/aistudio/PaddleClas/params /home/aistudio/PaddleClas/models/ResNeXt101_32x8d_wsl/params

mv: 无法获取'/home/aistudio/PaddleClas/model' 的文件状态(stat): 没有那个文件或目录
mv: 无法获取'/home/aistudio/PaddleClas/params' 的文件状态(stat): 没有那个文件或目录


In [18]:
# 预测
!python tools/infer/predict.py \
    -m "./models/ResNeXt101_32x8d_wsl/model" \
    -p "./models/ResNeXt101_32x8d_wsl/params" \
    -i "/home/aistudio/work/datasets/test/10.jpg" \
    --use_gpu=0 \
    --use_tensorrt=True 

gpu:  0
E1118 22:01:22.467455  9706 analysis_config.cc:231] To use TensorRT engine, please call EnableGpu() first
2021-11-18 22:01:24,291-INFO: class: 12
2021-11-18 22:01:24,291-INFO: score: 0.7409507036209106


# 六、开始搭建完整的预测部分

In [24]:
# 图像数据处理方法  -- 来自util.py
import cv2
import numpy as np


class DecodeImage(object):
    def __init__(self, to_rgb=True):
        self.to_rgb = to_rgb

    def __call__(self, img):
        data = np.frombuffer(img, dtype='uint8')
        img = cv2.imdecode(data, 1)
        if self.to_rgb:
            assert img.shape[2] == 3, 'invalid shape of image[%s]' % (
                img.shape)
            img = img[:, :, ::-1]

        return img


class ResizeImage(object):
    def __init__(self, resize_short=None):
        self.resize_short = resize_short

    def __call__(self, img):
        img_h, img_w = img.shape[:2]
        percent = float(self.resize_short) / min(img_w, img_h)
        w = int(round(img_w * percent))
        h = int(round(img_h * percent))
        return cv2.resize(img, (w, h))


class CropImage(object):
    def __init__(self, size):
        if type(size) is int:
            self.size = (size, size)
        else:
            self.size = size

    def __call__(self, img):
        w, h = self.size
        img_h, img_w = img.shape[:2]
        w_start = (img_w - w) // 2
        h_start = (img_h - h) // 2

        w_end = w_start + w
        h_end = h_start + h
        return img[h_start:h_end, w_start:w_end, :]


class NormalizeImage(object):
    def __init__(self, scale=None, mean=None, std=None):
        self.scale = np.float32(scale if scale is not None else 1.0 / 255.0)
        mean = mean if mean is not None else [0.485, 0.456, 0.406]
        std = std if std is not None else [0.229, 0.224, 0.225]

        shape = (1, 1, 3)
        self.mean = np.array(mean).reshape(shape).astype('float32')
        self.std = np.array(std).reshape(shape).astype('float32')

    def __call__(self, img):
        return (img.astype('float32') * self.scale - self.mean) / self.std


class ToTensor(object):
    def __init__(self):
        pass

    def __call__(self, img):
        img = img.transpose((2, 0, 1))
        return img

# 七、批量预测引擎的部署

In [40]:
# 预测引擎的构建 -- 借鉴predict.py
from paddle.fluid.core import AnalysisConfig
from paddle.fluid.core import create_paddle_predictor
import numpy as np

batch_size = 1
enable_benchmark = True
use_gpu = True
ir_optim = True
use_tensorrt = False  # 不能加速，否则会报错--tensorrt没有注册点，解决方法还没查找到
use_fp16 = True

# 创建引擎
def create_predictor(model_file, params_file, gpu_mem=8000):
    config = AnalysisConfig(model_file, params_file)

    if use_gpu:
        config.enable_use_gpu(gpu_mem, 0)
    else:
        config.disable_gpu()
    
    gpu_id = config.gpu_device_id()
    print('gpu: ',  gpu_id)

    config.disable_glog_info()
    config.switch_ir_optim(ir_optim)  # default true

    if use_tensorrt:
        config.enable_tensorrt_engine(
            precision_mode=AnalysisConfig.Precision.Half
            if use_fp16 else AnalysisConfig.Precision.Float32,
            max_batch_size=batch_size
            )  # use_calib_mode = True

    config.enable_mkldnn()

    config.enable_memory_optim()
    # use zero copy
    config.switch_use_feed_fetch_ops(False)
    predictor = create_paddle_predictor(config)

    return predictor


# 创建图像处理迭代器
def create_operators():
    size = 224
    img_mean = [0.485, 0.456, 0.406]
    img_std = [0.229, 0.224, 0.225]
    img_scale = 1.0 / 255.0

    decode_op = DecodeImage()
    resize_op = ResizeImage(resize_short=256)
    crop_op = CropImage(size=(size, size))
    normalize_op = NormalizeImage(
        scale=img_scale, mean=img_mean, std=img_std)
    totensor_op = ToTensor()

    return [decode_op, resize_op, crop_op, normalize_op, totensor_op]


# 传入图片文件、并通过迭代处理器进行图像处理
def preprocess(fname, ops):
    data = open(fname, 'rb').read()
    for op in ops:
        data = op(data)

    return data


# 预测引擎主函数
def main(model_file, aparams_file):
    
    # 验证参数 -- 不用取消注释，不影响使用
    # if not enable_benchmark:
    #     assert batch_size == 1
    #     assert use_fp16 is False
    # else:
    #     assert use_gpu is True
    #     assert use_tensorrt is True
    # # HALF precission predict only work when using tensorrt
    # if use_fp16 is True:
    #     assert use_tensorrt is True

    # 预测结果保存
    result = []

    operators = create_operators()  # 处理迭代器
    predictor = create_predictor(model_file, aparams_file)  # 预测引擎
    
    # 预测输入组件
    input_names = predictor.get_input_names()
    input_tensor = predictor.get_input_tensor(input_names[0])  # 输入参数的接口
    
    # 预测输出组件
    output_names = predictor.get_output_names()
    output_tensor = predictor.get_output_tensor(output_names[0])  # 输出参数的接口

    # 数据传入进行预测  -- 在这里开始批量预测的循环设置 -- 这里改成自己合适的数据集传入即可 -- 注意batch_size为1
    datasets_lens = 800    
    show_predect = True  # 是否展示与预测结果
    for i in range(datasets_lens):
        image_file = '/home/aistudio/work/datasets/test/{0}.jpg'.format(i)  # 次数 正好对应 id

        inputs = preprocess(image_file, operators)  # 数据处理

        inputs = np.expand_dims(
            inputs, axis=0).repeat(
                batch_size, axis=0).copy()  # 数据批转化

        input_tensor.copy_from_cpu(inputs)  # 复制数据到输入接口里

        predictor.zero_copy_run()   # 运行预测引擎

        output = output_tensor.copy_to_cpu()  # 获取输出接口的数据
        output = output.flatten()  # 将数据展平 -- 由于batch_size为1， 所以，直接展品后进行argmax操作会更方便
        cls = np.argmax(output)  # 得到类别
        score = output[cls]     # 得到对应的得分
        
        result.append(cls)
        # 可将每次的cls放入一个列表中返回，得到结果，然后保存或者其他操作

        # print('cls: ', cls)
        # print('score: ', score)
    
    print('Detection has done!')
    return result  # 所有预测的结果


In [41]:
# 执行预测器 -- 得到结果，返回result
model_path = './models/ResNeXt101_32x8d_wsl/model'  # 根据需要自行修改模型路径
param_path = './models/ResNeXt101_32x8d_wsl/params'
result = main(model_path, param_path)

gpu:  0


E1118 22:13:52.944423   174 paddle_pass_builder.cc:139] GPU not support MKLDNN yet
E1118 22:13:52.944468   174 paddle_pass_builder.cc:139] GPU not support MKLDNN yet
E1118 22:13:52.944620   174 paddle_pass_builder.cc:139] GPU not support MKLDNN yet


Detection has done!


In [47]:
# 将结果拼接好，保存到csv文件中
result = np.array(result).reshape(-1, 1)
indexs = np.array(range(800)).reshape(-1, 1)

save_datas = np.concatenate([indexs, result], axis=-1)
savedfs = pds.DataFrame(save_datas)
savedfs.to_csv('/home/aistudio/work/key.csv', header=None, index=None)  # 保存路径

#  训练-预测结束

In [48]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 102kB 4.8MB/s ta 0:00:011


In [ ]:
# 下载clas--如果下载不是很快的，建议上传--有该文件夹之后不用下载解压
!mkdir /home/aistudio/PaddleClas
!cd /home/aistudio/PaddleClas
!git clone https://github.com/PaddlePaddle/PaddleClas.git
# 解压clas压缩包
!unzip /home/aistudio/PaddleClas/paddlepaddle-PaddleClas-master.zip

In [50]:
# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')